### LangChain for LLM Application development

### LangChain for LLM application Development

It's a open source development framework for building LLM applications.  

Python and JavaScript packages.

Key value adds:
1. Modular components
2. Use cases: Common ways to combine components

Common components of LangChain:
1. Models
    * LLMs
    * Chat models
    * Text embedding models
2. Prompts
    * Prompt templates
    * Output Parsers
    * Example Selectors
3. Indexes - Ingesting data to combine with models
    * Document loaders
    * Text Splitters
    * Vector stores
    * Retrievers
4. Chains - End to end use cases
    * Prompt + LLm + Output parsing
    * Used as building blocks for longer chains
5. Agents - Use the Model as a *reasoning engine*
    * Agent Types
    * Agent Toolkits

### LangChain Models, Prompts and Parsers

Steps:
1. Install LangChain
2. Specify the use of the model
3. You build prompt templates in LangChain, this is useful to be able to reuse the prompt. 

In [ ]:
#%pip install langchain
#%pip install langchain_community #deprecated by the community
#%pip install -U langchain-ollama

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from langchain_community.chat_models import ChatOllama

llm_model = 'llama3:8b'
chat = ChatOllama(temperature=0.0, model=llm_model)
chat

C:\Users\zanes\AppData\Local\Temp\ipykernel_27140\2294417168.py:4: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  chat = ChatOllama(temperature=0.0, model=llm_model)


ChatOllama(model='llama3:8b', temperature=0.0)

In [4]:
#Now for prompt template
template_string = """Translate the text that is delimited by the triple backticks into a style that is {style}. \
    text: ```{text}```"""

In [5]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['style', 'text'], input_types={}, partial_variables={}, template='Translate the text that is delimited by the triple backticks into a style that is {style}.     text: ```{text}```')

In [6]:
prompt_template.messages[0].prompt.input_variables

['style', 'text']

In [7]:
customer_style = """American English\
    in a calm and respectful tone"""

In [8]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [9]:
customer_messages = prompt_template.format_messages(style=customer_style, text=customer_email)
print(type(customer_messages))
print(type(customer_messages[0]))
print(customer_messages)

<class 'list'>
<class 'langchain_core.messages.human.HumanMessage'>
[HumanMessage(content="Translate the text that is delimited by the triple backticks into a style that is American English    in a calm and respectful tone.     text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```", additional_kwargs={}, response_metadata={})]


In [10]:
#Calling the llm to translate
customer_response = chat(customer_messages)

C:\Users\zanes\AppData\Local\Temp\ipykernel_27140\4281539771.py:2: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  customer_response = chat(customer_messages)


In [11]:
print(customer_response.content)

Here is the translation:

"I'm extremely frustrated that my blender lid came loose and splattered my kitchen walls with smoothie! To make things even more disappointing, the warranty doesn't cover the cost of cleaning up the mess in my kitchen. I really need your help right now."

I've maintained a calm and respectful tone while translating the text from pirate speak to American English. Let me know if you have any further requests!


In [12]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""
service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""
service_messages = prompt_template.format_messages(style = service_style_pirate, text = service_reply)

print(service_messages[0].content)

Translate the text that is delimited by the triple backticks into a style that is a polite tone that speaks in English Pirate.     text: ```Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!
```


In [13]:
#Calling the llm to translate again
service_response = chat(service_messages)
print(service_response.content)

Arrrr, me hearty!

`Ahoy, valued patron! Yer warranty don't cover them tidyin' costs fer yer galley, matey. It seems ye got yerself into a spot o' trouble by forgettin' to slap on the lid afore startin' up yer blender, savvy? Well, shiver me timbers! Ye gotta take responsibility fer yer own mistakes, mate. No use cryin' over spilled grog now, is there? So, it's back to scrubbin' away yerself, me hearty! Fair winds and following seas!`


**LangChain prompts can be engineered to do tasks such as summarization, connecting to databases etc.**

### Output Parsing

One other aspect of LangChain libraries is it supports output pausing - This means that the LLM can 

When building a complex application using an LLM you often instruct the LLM to output in a certain format e.g. if you do **chain of thought**:
1. Thought - Gives LLM space to think
2. Action - to do the action
3. Observation - what it learned from the action
4. ....
5. Thought again
6. Final action

A **Parser** can interpret the output that the LLM gives

In [15]:
# Desired output
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

{'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable!'}

In [ ]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

#Gives the instruction to ingest the customer review, and extract the 3 values (seen in the example above) and output in a JSON

review_template = """\ 
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [17]:
#from langchain.promtps import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n'), additional_kwargs={})]


In [18]:
messages = prompt_template.format_messages(text=customer_review)
response = chat(messages)
print(response.content)

Here is the extracted information in JSON format:

```
{
    "gift": True,
    "delivery_days": 2,
    "price_value": ["It's slightly more expensive than the other leaf blowers out there"]
}
```


In [19]:
type(response.content)

str

In [ ]:
#Gives an error because it is a string not a JSON
response.content.get('gift')

AttributeError: 'str' object has no attribute 'get'

In [21]:
#Parsing the LLM to a python dictionary
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]

In [23]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.
	"delivery_days": string  // How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.
	"price_value": string  // Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.
}
```


In [67]:
# Do a second review
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}

IMPORTANT: Output only valid JSON. Do not add any text, instructions, or Markdown code fences. Boolean values must be lowercase (true/false).

"""

prompt = ChatPromptTemplate.from_template(template=review_template_2) #Initialize the prompt
messages = prompt.format_messages(text=customer_review, format_instructions=format_instructions) #Format the message

In [68]:
print(messages[0].content)

For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

text: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```

In [69]:
response = chat(messages)
print(response.content)

```json
{
	"gift": "True",
	"delivery_days": "2",
	"price_value": ["It's slightly more expensive than the other leaf blowers out there"]
}


In [70]:
output_dict = output_parser.parse(response.content)
output_dict

{'gift': 'True',
 'delivery_days': '2',
 'price_value': ["It's slightly more expensive than the other leaf blowers out there"]}

In [71]:
type(output_dict)

dict

In [72]:
output_dict.get('delivery_days')

'2'